In [1]:
import pickle
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# import scienceplots
from tqdm import tqdm
from shapely import LineString, Point
from seaborn import displot, histplot, relplot
plt.style.use(['seaborn-v0_8-paper'])
# plt.style.use(['ieee', 'notebook'])

from waymo_devkit.conflict import Conflict

In [2]:
with open('./pkl/conflict_pet15s.pkl', 'rb') as file:
    conflict_list_from_pickle = pickle.load(file)
print(f"#Conflicts: {len(conflict_list_from_pickle)}")

conflict_list = list()
for c in conflict_list_from_pickle:
    conflict_list.append(
        Conflict(
            leader_id=c['leader_id'],
            leader_index=c['leader_index'],
            leader_states=c['leader_states'],
            leader_time_at_conflict=c['leader_time_at_conflict'],
            follower_id=c['follower_id'],
            follower_index=c['follower_index'],
            follower_states=c['follower_states'],
            follower_time_at_conflict=c['follower_time_at_conflict'],
            PET=c['PET'],
            conflict_type=c['conflict_type'],
            leader_is_av=c['leader_is_av'],
            follower_is_av=c['follower_is_av'],
    ))

#Conflicts: 889


/home/gavin/waymo/waymo_devkit/conflict.py:149: RuntimeWarning: divide by zero encountered in double_scalars
  TTCs.append(follower_trajectory_length / (self.follower_beforeConflict_speeds[follower_index]))
/home/gavin/waymo/waymo_devkit/conflict.py:164: RuntimeWarning: divide by zero encountered in double_scalars
  follower_trajectory_length / self.follower_beforeConflict_speeds[follower_index] - \
/home/gavin/waymo/waymo_devkit/conflict.py:165: RuntimeWarning: divide by zero encountered in double_scalars
  leader_trajectory_length / self.leader_beforeConflict_speeds[leader_index]


In [26]:
"""
Construct a dataframe with the following columns:
- Conflict type
- Vehicle order
- PET
"""
conflict_cols_in_df = [
    "Conflict type",
    "Vehicle order",
    "leader_type",
    "follower_type",
    "PET",
    "TA",
]

# generate each rows in the dataframe
conflict_rows_in_df = list()
for index, conflict in tqdm(enumerate(conflict_list)):

    leader_type = "AV" if conflict.leader_is_av else "HV"
    follower_type = "AV" if conflict.follower_is_av else "HV" 
    conflict_type = "Cross" if conflict.conflict_type == "CROSS" else "Merge"

    TAs = conflict.time_advantages
    if len(TAs) > 0:
        conflict_rows_in_df.append([
            conflict_type,
            leader_type+"-"+follower_type,
            conflict.leader_is_av,
            conflict.follower_is_av,
            conflict.PET, 
            TAs[0],
        ])
conflict_df = pd.DataFrame(conflict_rows_in_df, columns=conflict_cols_in_df)

# Post-process
PET_THRESHOLD = 10
conflict_df = conflict_df[conflict_df["PET"] <= PET_THRESHOLD]

# display info
conflict_df.info()

889it [00:00, 58220.57it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 843 entries, 0 to 863
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Conflict type  843 non-null    object 
 1   Vehicle order  843 non-null    object 
 2   leader_type    843 non-null    bool   
 3   follower_type  843 non-null    bool   
 4   PET            843 non-null    float64
 5   TA             843 non-null    float64
dtypes: bool(2), float64(2), object(2)
memory usage: 34.6+ KB


In [28]:
HV_numerator, HV_denominator = 0, 0
AV_numerator, AV_denominator = 0, 0

for index, row in conflict_df.iterrows():
    # print(row.TA)
    if (row.TA > 0 and row.leader_type) or (row.TA < 0 and row.follower_type):
        # this means AV has the time advantage 
        AV_denominator += 1
        if conflict.leader_type:
            AV_numerator += 1
    elif (row.TA > 0 and not row.leader_type) or (row.TA < 0 and not row.follower_type):
        # this means HV has the time advantage 
        HV_denominator += 1
        if not conflict.leader_type:
            HV_numerator += 1

print(f"AV: {AV_numerator}, {AV_denominator}")
print(f"HV: {HV_numerator}, {HV_denominator}") 
print(f"AV: {AV_numerator / AV_denominator}")
print(f"HV: {HV_numerator / HV_denominator}")

AV: 134, 134
HV: 0, 709
AV: 1.0
HV: 0.0
